In [1]:
import redis


In [2]:
r1=redis.StrictRedis(db=1)
r2=redis.StrictRedis(db=2)

N=10

In [11]:
import random

def random_variant():
#     r=random.sample(["A","T","C","G"],1)[0]
    r="A"
    a=random.sample(["A","T","C","G"],1)[0]
    pos=random.randint(0,100000)
    v= "{ref}{pos}{alt}".format(ref=r,pos=pos,alt=a)
    return v

def generate_variant_set():
    n=random.randint(1000,10000)
    vars=set()
    for i in range(n):
        v=random_variant()
        vars.add(v)
    return vars
sample_to_variants={}
for s in range(N):
    if (s % 100)==0:
        print(s)
    sample_to_variants[s]=generate_variant_set()


0


In [12]:
%%time
def insert(s,variants,r=r1):
    pipe=r.pipeline()
    for v in variants:
        pipe.sadd(s,v)
    pipe.execute()

CPU times: user 5 µs, sys: 4 µs, total: 9 µs
Wall time: 6.91 µs


In [19]:
%%time

def insert_variants(variants, unique=True,r=r2):
    if unique:
        pipe=r.pipeline()
        for v in variants:
            i=r.incr("num_variants")
            pipe.hset("variants",v,i)               
        pipe.execute()
    else:
        for v in variants:
            if r.hexists("variants",v):
                i=r.hget("variants", v)
            else:
                i=r.incr("num_variants")
                r.hset("variants",v,i)        
    
def insert2(s,variants,r=r2):
    r.sadd("samples",s)
    print(s)
    pipe=r.pipeline()
    for v in variants:
        pipe.hget("variants", v)
    res=pipe.execute()
    indexes=res
    pipe=r.pipeline()
    for i in indexes:
        pipe.setbit(s,i,1)
    pipe.execute()
        
#     pipe=r.pipeline()
#     for v in variants:
#         pipe.sadd(s,v)
#     pipe.execute()
r1.flushall()
r2.flushall()

# insert2(1,sample_to_variants[1])
all_variants=set()
for sample,variants in sample_to_variants.items():
    all_variants.update(variants)
print(len(all_variants))
insert_variants(all_variants)
for sample,variants in sample_to_variants.items():
    insert2(sample,variants)
    insert(sample,variants)    

59579
0
1
2
3
4
5
6
7
8
9
CPU times: user 6.43 s, sys: 851 ms, total: 7.28 s
Wall time: 8.68 s


In [14]:
%%time
def distance(sample,r=r1):
    d={}
    for query_sample in range(N):
        if query_sample != sample:
            s1=r.sdiff(sample,query_sample)
            s2=r.sdiff(query_sample,sample)
            d[query_sample]=len(s1.union(s2))    
    return d
# distance(1)    

def distance2(sample,r=r2):
    pipe=r.pipeline()
    for query_sample in range(N):
        if query_sample != sample:
            pipe.bitop("xor","_".join([str(sample),str(query_sample)]),sample,query_sample)
    pipe.execute()
    
    pipe=r.pipeline()
    for query_sample in range(N):
        if query_sample != sample:
            pipe.bitcount("_".join([str(sample),str(query_sample)]))
    res=pipe.execute()

    d={}
    for q,diff in zip(range(N),res):
        d[q]=diff
    
    return (d)

def distance3(sample):
    d={}
    for query_sample,variants in sample_to_variants.items():
        if query_sample != sample:
            d[query_sample]=len(sample_to_variants[sample].symmetric_difference(variants))
    return d
            

CPU times: user 16 µs, sys: 1e+03 ns, total: 17 µs
Wall time: 21.9 µs


In [15]:
%%time
result=distance(1)  

CPU times: user 469 ms, sys: 6.64 ms, total: 475 ms
Wall time: 609 ms


In [16]:
%%time
result=distance2(1)

CPU times: user 867 µs, sys: 856 µs, total: 1.72 ms
Wall time: 1.14 ms


In [17]:
%%time
result=distance3(1)

CPU times: user 17.1 ms, sys: 552 µs, total: 17.6 ms
Wall time: 32.6 ms


In [18]:
print(distance(1))
print(distance2(3))
print(distance3(1))

{0: 15457, 2: 11196, 3: 16134, 4: 15530, 5: 7639, 6: 12188, 7: 8344, 8: 16005, 9: 12956}
{0: 18431, 1: 16134, 2: 14178, 3: 18474, 4: 10721, 5: 15168, 6: 11388, 7: 18965, 8: 15946}
{0: 15457, 2: 11196, 3: 16134, 4: 15530, 5: 7639, 6: 12188, 7: 8344, 8: 16005, 9: 12956}
